# pandas.pivot_table

## Agenda

- `pandas.pivot_table`の挙動確認
- pivot_tableと`aggfunc`の設定


In [1]:
!Python -V

Python 3.7.4


### Import

In [2]:
import pandas as pd
import numpy as np

### pandas verision

In [3]:
pd.__version__

'1.0.3'

## 1. Pivot Tableの作成

In [4]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                          "bar", "bar", "bar", "bar"],
                    "B": ["one", "one", "one", "two", "two",
                          "one", "one", "two", "two"],
                    "C": ["small", "large", "large", "small",
                          "small", "large", "small", "small",
                          "large"],
                    "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                    "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [5]:
pd.pivot_table(df, values='D', index=['A', 'B'],
                     columns=['C'], aggfunc=np.sum)

C        large  small
A   B                
bar one    4.0    5.0
    two    7.0    6.0
foo one    4.0    1.0
    two    NaN    6.0

In [6]:
pivot_df = pd.pivot_table(df, values='D', index=['A', 'B'], 
                          columns=['C'], aggfunc=np.sum, fill_value = 0)
pivot_df

C        large  small
A   B                
bar one      4      5
    two      7      6
foo one      4      1
    two      0      6

In [7]:
pd.pivot_table(df, values='D', index=['A', 'B'],
                     columns=['C'], aggfunc= lambda x: len(np.unique(x)))

C        large  small
A   B                
bar one    1.0    1.0
    two    1.0    1.0
foo one    1.0    1.0
    two    NaN    1.0

In [8]:
table = pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean,
                             'E': [min, max, np.mean]})
table    

D    E               
               mean  max      mean  min
A   C                                  
bar large  5.500000  9.0  7.500000  6.0
    small  5.500000  9.0  8.500000  8.0
foo large  2.000000  5.0  4.500000  4.0
    small  2.333333  6.0  4.333333  2.0

### Multiindexを用いたpivot tableの作成

In [9]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
                     'foo', 'foo', 'qux', 'qux'],
                    ['one', 'two', 'one', 'two',
                     'one', 'two', 'one', 'two']]))

index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
index

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('baz', 'one'),
            ('baz', 'two'),
            ('foo', 'one'),
            ('foo', 'two'),
            ('qux', 'one'),
            ('qux', 'two')],
           names=['first', 'second'])

In [10]:
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=['A', 'B'])
df

A         B
first second                    
bar   one     1.108537 -0.945213
      two    -1.247946 -0.749161
baz   one     0.649141  1.408186
      two    -1.837249  0.586930
foo   one     0.261689  0.266917
      two    -0.695311  0.185367
qux   one    -0.501698  0.714050
      two    -0.853868 -0.080148

### stackを用いた変換

In [11]:
stacked = df.stack()
stacked

first  second   
bar    one     A    1.108537
               B   -0.945213
       two     A   -1.247946
               B   -0.749161
baz    one     A    0.649141
               B    1.408186
       two     A   -1.837249
               B    0.586930
foo    one     A    0.261689
               B    0.266917
       two     A   -0.695311
               B    0.185367
qux    one     A   -0.501698
               B    0.714050
       two     A   -0.853868
               B   -0.080148
dtype: float64

In [12]:
type(stacked)

pandas.core.series.Series

### unstack

In [13]:
stacked.unstack()

A         B
first second                    
bar   one     1.108537 -0.945213
      two    -1.247946 -0.749161
baz   one     0.649141  1.408186
      two    -1.837249  0.586930
foo   one     0.261689  0.266917
      two    -0.695311  0.185367
qux   one    -0.501698  0.714050
      two    -0.853868 -0.080148

In [14]:
stacked.unstack(0)

first          bar       baz       foo       qux
second                                          
one    A  1.108537  0.649141  0.261689 -0.501698
       B -0.945213  1.408186  0.266917  0.714050
two    A -1.247946 -1.837249 -0.695311 -0.853868
       B -0.749161  0.586930  0.185367 -0.080148

In [15]:
stacked.unstack(1)

second        one       two
first                      
bar   A  1.108537 -1.247946
      B -0.945213 -0.749161
baz   A  0.649141 -1.837249
      B  1.408186  0.586930
foo   A  0.261689 -0.695311
      B  0.266917  0.185367
qux   A -0.501698 -0.853868
      B  0.714050 -0.080148

### Pivot Tableの読み込みから変換

In [16]:
print(df)
df.to_excel('pivot_table.xlsx')

                     A         B
first second                    
bar   one     1.108537 -0.945213
      two    -1.247946 -0.749161
baz   one     0.649141  1.408186
      two    -1.837249  0.586930
foo   one     0.261689  0.266917
      two    -0.695311  0.185367
qux   one    -0.501698  0.714050
      two    -0.853868 -0.080148


In [17]:
df_excel = pd.read_excel('pivot_table.xlsx')
df_excel

,first,second,A,B
0,bar,one,1.108537,-0.945213
1,NaN,two,-1.247946,-0.749161
2,baz,one,0.649141,1.408186
3,NaN,two,-1.837249,0.586930
4,foo,one,0.261689,0.266917
5,NaN,two,-0.695311,0.185367
6,qux,one,-0.501698,0.714050
7,NaN,two,-0.853868,-0.080148


In [18]:
df_excel.fillna(method='ffill')

,first,second,A,B
0,bar,one,1.108537,-0.945213
1,bar,two,-1.247946,-0.749161
2,baz,one,0.649141,1.408186
3,baz,two,-1.837249,0.586930
4,foo,one,0.261689,0.266917
5,foo,two,-0.695311,0.185367
6,qux,one,-0.501698,0.714050
7,qux,two,-0.853868,-0.080148


## 2. sales pipeline (funnel) 分析を実施する
　
sales pipeline分析における代表的なquestionは

1. How much revenue is in the pipeline?
2. What products are in the pipeline?
3. Who has what products at what stage?
4. How likely are we to close deals by year end?


In [19]:
### dataset
PATH = './data/sales-funnel.xlsx'
df = pd.read_excel(PATH)
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


### 前処理

- the status columnを`category`に変換

In [20]:
df["Status"] = df["Status"].astype("category")
df["Status"].cat.set_categories(["won","pending","presented","declined"],inplace=True)

In [21]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=np.sum)

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

In [22]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=[np.mean,len])

mean   len
                                    Price Price
Manager       Rep                              
Debra Henley  Craig Booker   20000.000000     4
              Daniel Hilton  38333.333333     3
              John Smith     20000.000000     2
Fred Anderson Cedric Moss    27500.000000     4
              Wendy Yule     44250.000000     4

In [23]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],
               columns=["Product"],aggfunc=[np.sum])

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

`fillna`の処理

In [24]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],
               columns=["Product"],aggfunc=[np.sum],fill_value=0)

sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

valueの追加

In [25]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price","Quantity"],
               columns=["Product"],aggfunc=[np.sum],fill_value=0)

sum                                        \
                              Price                              Quantity   
Product                         CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                           
Debra Henley  Craig Booker    65000        5000       0    10000        2   
              Daniel Hilton  105000           0       0    10000        4   
              John Smith      35000        5000       0        0        1   
Fred Anderson Cedric Moss     95000        5000       0    10000        3   
              Wendy Yule     165000        7000    5000        0        7   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            2       0        1  
              Daniel Hilton           0       0        1  
              John Smith              2       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              3       2        0

In [26]:
pd.pivot_table(df,index=["Manager","Rep","Product"],
               values=["Price","Quantity"],aggfunc=[np.sum],fill_value=0)

sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2

In [27]:
pd.pivot_table(df,index=["Manager","Rep","Product"],
               values=["Price","Quantity"],
               aggfunc=[np.sum,np.mean],fill_value=0,margins=True)

sum                   mean  \
                                          Price Quantity         Price   
Manager       Rep           Product                                      
Debra Henley  Craig Booker  CPU           65000        2  32500.000000   
                            Maintenance    5000        2   5000.000000   
                            Software      10000        1  10000.000000   
              Daniel Hilton CPU          105000        4  52500.000000   
                            Software      10000        1  10000.000000   
              John Smith    CPU           35000        1  35000.000000   
                            Maintenance    5000        2   5000.000000   
Fred Anderson Cedric Moss   CPU           95000        3  47500.000000   
                            Maintenance    5000        1   5000.000000   
                            Software      10000        1  10000.000000   
              Wendy Yule    CPU          165000        7  82500.000000   
                            Maintenance    7000        3   7000.000000   
                            Monitor        5000        2   5000.000000   
All                                      522000       30  30705.882353   

                                                   
                                         Quantity  
Manager       Rep           Product                
Debra Henley  Craig Booker  CPU          1.000000  
                            Maintenance  2.000000  
                            Software     1.000000  
              Daniel Hilton CPU          2.000000  
                            Software     1.000000  
              John Smith    CPU          1.000000  
                            Maintenance  2.000000  
Fred Anderson Cedric Moss   CPU          1.500000  
                            Maintenance  1.000000  
                            Software     1.000000  
              Wendy Yule    CPU          3.500000  
                            Maintenance  3.000000  
                            Monitor      2.000000  
All                                      1.764706

In [28]:
pd.pivot_table(df,index=["Manager","Status"],values=["Price"],
               aggfunc=[np.sum],fill_value=0,margins=True)

sum
                          Price
Manager       Status           
Debra Henley  won         65000
              pending     50000
              presented   50000
              declined    70000
Fred Anderson won        172000
              pending      5000
              presented   45000
              declined    65000
All                      522000

### `dict`を用いて集計関数を指定

In [29]:
pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc={"Quantity":len,"Price":np.sum},fill_value=0)

Price                              Quantity  \
Product                     CPU Maintenance Monitor Software      CPU   
Manager       Status                                                    
Debra Henley  won         65000           0       0        0        1   
              pending     40000       10000       0        0        1   
              presented   30000           0       0    20000        1   
              declined    70000           0       0        0        2   
Fred Anderson won        165000        7000       0        0        2   
              pending         0        5000       0        0        0   
              presented   30000           0    5000    10000        1   
              declined    65000           0       0        0        1   

                                                      
Product                 Maintenance Monitor Software  
Manager       Status                                  
Debra Henley  won                 0       0        0  
              pending             2       0        0  
              presented           0       0        2  
              declined            0       0        0  
Fred Anderson won                 1       0        0  
              pending             1       0        0  
              presented           0       1        1  
              declined            0       0        0

In [30]:
table = pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc={"Quantity":len,"Price":[np.sum,np.mean]},fill_value=0)
table

Price                                       \
                          mean                                  sum   
Product                    CPU Maintenance Monitor Software     CPU   
Manager       Status                                                  
Debra Henley  won        65000           0       0        0   65000   
              pending    40000        5000       0        0   40000   
              presented  30000           0       0    10000   30000   
              declined   35000           0       0        0   70000   
Fred Anderson won        82500        7000       0        0  165000   
              pending        0        5000       0        0       0   
              presented  30000           0    5000    10000   30000   
              declined   65000           0       0        0   65000   

                                                     Quantity              \
                                                          len               
Product                 Maintenance Monitor Software      CPU Maintenance   
Manager       Status                                                        
Debra Henley  won                 0       0        0        1           0   
              pending         10000       0        0        1           2   
              presented           0       0    20000        1           0   
              declined            0       0        0        2           0   
Fred Anderson won              7000       0        0        2           1   
              pending          5000       0        0        0           1   
              presented           0    5000    10000        1           0   
              declined            0       0        0        1           0   

                                          
                                          
Product                 Monitor Software  
Manager       Status                      
Debra Henley  won             0        0  
              pending         0        0  
              presented       0        2  
              declined        0        0  
Fred Anderson won             0        0  
              pending         0        0  
              presented       1        1  
              declined        0        0

### Pivot Tableのフィルタリング

In [31]:
table.query('Manager == ["Debra Henley"]')

Price                                                  \
                         mean                                 sum               
Product                   CPU Maintenance Monitor Software    CPU Maintenance   
Manager      Status                                                             
Debra Henley won        65000           0       0        0  65000           0   
             pending    40000        5000       0        0  40000       10000   
             presented  30000           0       0    10000  30000           0   
             declined   35000           0       0        0  70000           0   

                                        Quantity                               
                                             len                               
Product                Monitor Software      CPU Maintenance Monitor Software  
Manager      Status                                                            
Debra Henley won             0        0        1           0       0        0  
             pending         0        0        1           2       0        0  
             presented       0    20000        1           0       0        2  
             declined        0        0        2           0       0        0

In [32]:
table.query('Status == ["pending","won"]')

Price                                                   \
                        mean                                  sum               
Product                  CPU Maintenance Monitor Software     CPU Maintenance   
Manager       Status                                                            
Debra Henley  won      65000           0       0        0   65000           0   
              pending  40000        5000       0        0   40000       10000   
Fred Anderson won      82500        7000       0        0  165000        7000   
              pending      0        5000       0        0       0        5000   

                                       Quantity                               
                                            len                               
Product               Monitor Software      CPU Maintenance Monitor Software  
Manager       Status                                                          
Debra Henley  won           0        0        1           0       0        0  
              pending       0        0        1           2       0        0  
Fred Anderson won           0        0        2           1       0        0  
              pending       0        0        0           1       0        0